In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# import yfinance as yf
import datetime

import pandas
# from pandas_datareader import data as pdr

from pyathena import connect
import pandas as pd

In [25]:
# Set up connection parameters
region_name = # change to your region
aws_access_key_id = # Change Accordingly
aws_secret_access_key = # Change Accordingly
s3_staging_dir =#  Change Accordingly
# Connect to Athena
conn = connect(region_name=region_name,
               aws_access_key_id=aws_access_key_id,
               aws_secret_access_key=aws_secret_access_key,
               s3_staging_dir=s3_staging_dir)

# Execute a SQL query
query = 'SELECT * FROM "kafka-stock-market"."kafka_stock_analysis_aakash" ORDER BY date'
data = pd.read_sql(query, conn)

# Print the results
print(len(data))
print(data.tail())

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
# prepare Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['close'].values.reshape(-1,1))
print(scaled_data)

prediction_days = 60

x_train = []
y_train = []

for x in range(prediction_days, len(scaled_data)):
  x_train.append(scaled_data[x-prediction_days:x, 0])
  y_train.append(scaled_data[x,0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


In [6]:
# build the model
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1)) # prediction of the next close

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=25, batch_size=32)

In [7]:
''' Test the model accuracy on existing data'''
test_data = data.tail(60)

actual_prices = test_data['close'].values

total_dataset = pd.concat((data['close'], test_data['close']), axis=0)

model_inputs = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inputs = model_inputs.reshape(-1,1)
model_inputs = scaler.transform(model_inputs)

/var/folders/t0/8mj1dc3j1wzdh4k57nbbcwg80000gn/T/ipykernel_21494/2242965143.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append([row_data], ignore_index=True)


KeyError: 'VarCharValue'

In [8]:
x_test = []

for x in range(prediction_days, len(model_inputs)):
  x_test.append(model_inputs[x-prediction_days:x, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1]))
predicted_prices = model.predict(x_test)
predicted_prices = scaler.inverse_transform(predicted_prices)

     |████████████████████████████████| 67 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 10.5 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 134 kB 34.9 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 32.0 MB/s eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.27.59
    Uninstalling botocore-1.27.59:
      Successfully uninstalled botocore-1.27.59
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.5.0
    Uninstalling s3transfer-0.5.0:
      Successfully uninstalled s3transfer-0.5.0
  Attempting uninstall: boto3
    Found existing installation: boto3 1.21.32
    Uninstalling boto3-1.21.32:
      Successfully uninstalled boto3-1.21.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.4.2 requires botocore<1.27.60,>=1.27.59, but you h

In [1]:
# plot the preductions
company = "Test"
plt.plot(actual_prices, color="black", label=f"Actual {company} Price")
plt.plot(predicted_prices, color="green", label=f"Predicted {company} Price")
plt.title(f"{company} Share Price")
plt.xlabel('Time')
plt.ylabel(f'{company} Share Price')
plt.legend()
plt.show()

/Users/aakgna/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Empty DataFrame
Columns: [index, date, open, high, low, close, adj close, volume, closeusd]
Index: []


In [24]:
# Predict next day 
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs+1), 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print(f"Prediction: {prediction}")

NameError: name 'MinMaxScaler' is not defined